# Import

In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import os
import math
from pysentimiento import create_analyzer
from transformers import pipeline


# Read Data

In [ ]:
import pandas as pd

# Read the main general tweets file (TSV format)
df = pd.read_csv('general_tweets.tsv', sep='\t')

# # Define the folder path and list of CSV file names
# output_folder = '/home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/'
# file_names = [
#     'job1_tweets_emotion_pysentimento.csv',
#     'job2_tweets_emotion_pysentimento.csv',
#     'job3_tweets_emotion_pysentimento.csv',
#     'job4_tweets_emotion_pysentimento.csv',
#     'job5_tweets_emotion_pysentimento.csv',
#     'job6_tweets_emotion_pysentimento.csv',
#     'job7_tweets_emotion_pysentimento.csv',
#     'job8_tweets_emotion_pysentimento.csv',
#     'job9_tweets_emotion_pysentimento.csv',
#     'job10_tweets_emotion_pysentimento.csv',
#     'job11_tweets_emotion_pysentimento.csv',
#     'job12_tweets_emotion_pysentimento.csv',
#     'job13_tweets_emotion_pysentimento.csv',
# ]
#
# # Read and concatenate all emotion CSVs into a single DataFrame
# emotion_dfs = [pd.read_csv(f"{output_folder}{file}") for file in file_names]
# general = pd.concat(emotion_dfs, ignore_index=True)


In [ ]:
# Remove rows where 'full_text' is NaN
df = df[df['full_text'].notna()]

# Remove rows where 'full_text' is an empty string or contains only whitespace
df = df[df['full_text'].str.strip() != '']


In [ ]:
# Reset index to ensure clean row access
df = df.reset_index(drop=True)

# Define constants
CHUNK_SIZE = 10_000  # Number of rows per chunk
OUTPUT_DIR = '/home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new'

# Initialize the emotion analyzer
emotion_analyzer = create_analyzer(task="emotion", lang="en")

# Loop over the DataFrame in chunks
for chunk_start in range(0, len(df), CHUNK_SIZE):
    chunk_end = chunk_start + CHUNK_SIZE
    chunk_df = df.iloc[chunk_start:chunk_end]

    processed_data = []

    for i, row in chunk_df.iterrows():
        if i % 1000 == 0:
            print(f'Processing row {i}')

        text = row['full_text']

        try:
            result = emotion_analyzer.predict(text)
            processed_data.append([
                row['screen_name'],
                row['created_at'],
                row['name'],
                row['uid'],
                str(row['id_str']),
                result.probas['joy'],
                result.probas['sadness'],
                result.probas['anger'],
                result.probas['surprise'],
                result.probas['disgust'],
                result.probas['fear'],
                result.probas['others'],
                text
            ])
        except Exception as e:
            # Optionally log the error
            continue

    # Create a DataFrame from processed rows
    result_df = pd.DataFrame(processed_data, columns=[
        'screen_name', 'created_at', 'name', 'uid', 'id_str',
        'joy_pys', 'sadness_pys', 'anger_pys', 'surprise_pys', 'disgust_pys', 'fear_pys',
        'others_pys', 'full_text'
    ])

    # Build output file path
    file_number = chunk_start // CHUNK_SIZE + 1
    output_path = os.path.join(OUTPUT_DIR, f'job{file_number}_tweets_emotion_pysentimento.csv')

    # Save to CSV (with header if first write)
    if os.path.exists(output_path):
        result_df.to_csv(output_path, mode='a', header=False, index=False)
    else:
        result_df.to_csv(output_path, index=False)

    print(f'Saved processed chunk to: {output_path}')


In [13]:
row['full_text']

nan